In [1]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.wkt import loads


In [14]:
api_key=''

In [2]:
CT_0=pd.read_csv('../Data/tracts_subway_bus_access.csv',index_col=0)
CT_0.set_index('geoid',inplace=True)
CT_0['geometry'] = CT_0['geometry'].apply(loads)

# Convert the pandas DataFrame to a GeoDataFrame
CT_0 = gpd.GeoDataFrame(CT_0, geometry='geometry', crs = '4326')
#filter this based on what subway access and bus count 
CT_0.head(n=5)

,ctlabel,boro_code,boro_name,ct2010,boro_ct2010,cdeligibil,ntacode,ntaname,puma,shape_leng,shape_area,geometry,Bus_Access,Subway_Access,Bus_Count,Subway_Count
geoid,,,,,,,,,,,,,,,,
36061009800,98.0,1,Manhattan,9800,1009800,I,MN19,Turtle Bay-East Midtown,3808,5534.200308,1.906016e+06,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ...",1,1,22,3
36061010200,102.0,1,Manhattan,10200,1010200,I,MN17,Midtown-Midtown South,3807,5687.802439,1.860993e+06,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ...",1,1,25,8
36061010400,104.0,1,Manhattan,10400,1010400,I,MN17,Midtown-Midtown South,3807,5693.036367,1.864600e+06,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ...",1,1,21,9
36061011300,113.0,1,Manhattan,11300,1011300,I,MN17,Midtown-Midtown South,3807,5699.860640,1.890907e+06,"MULTIPOLYGON (((-73.98412 40.75484, -73.98460 ...",1,1,21,14
36061013000,130.0,1,Manhattan,13000,1013000,I,MN40,Upper East Side-Carnegie Hill,3805,5807.972956,1.918145e+06,"MULTIPOLYGON (((-73.96148 40.77432, -73.96194 ...",1,1,12,2


In [3]:
print(len(CT_0))

2055


In [4]:
CT_0=CT_0[(CT_0['Subway_Access']==1)|(CT_0['Bus_Access']==1)]#remove census tracts with no public transit access

In [5]:
print(len(CT_0))

2041


In [6]:
CT_0['centroid'] = CT_0.to_crs('+proj=cea').centroid.to_crs(4326)
CT_0['Center_Lat'] = CT_0['centroid'].y
CT_0['Center_Lon'] = CT_0['centroid'].x
CT_0 = CT_0.drop(columns = 'centroid')
CT_0 = CT_0.to_crs(32118)
CT_0['area_km2'] = CT_0.geometry.area / 1e6
CT_0 = CT_0.to_crs(4326)
CT_0.head()

,ctlabel,boro_code,boro_name,ct2010,boro_ct2010,cdeligibil,ntacode,ntaname,puma,shape_leng,shape_area,geometry,Bus_Access,Subway_Access,Bus_Count,Subway_Count,Center_Lat,Center_Lon,area_km2
geoid,,,,,,,,,,,,,,,,,,,
36061009800,98.0,1,Manhattan,9800,1009800,I,MN19,Turtle Bay-East Midtown,3808,5534.200308,1.906016e+06,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ...",1,1,22,3,40.755788,-73.967775,0.177075
36061010200,102.0,1,Manhattan,10200,1010200,I,MN17,Midtown-Midtown South,3807,5687.802439,1.860993e+06,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ...",1,1,25,8,40.759424,-73.974449,0.172893
36061010400,104.0,1,Manhattan,10400,1010400,I,MN17,Midtown-Midtown South,3807,5693.036367,1.864600e+06,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ...",1,1,21,9,40.760779,-73.977673,0.173228
36061011300,113.0,1,Manhattan,11300,1011300,I,MN17,Midtown-Midtown South,3807,5699.860640,1.890907e+06,"MULTIPOLYGON (((-73.98412 40.75484, -73.98460 ...",1,1,21,14,40.754775,-73.987870,0.175672
36061013000,130.0,1,Manhattan,13000,1013000,I,MN40,Upper East Side-Carnegie Hill,3805,5807.972956,1.918145e+06,"MULTIPOLYGON (((-73.96148 40.77432, -73.96194 ...",1,1,12,2,40.772736,-73.964734,0.178202


In [19]:
# CT_centroids=gpd.read_file('../Data/NYC_tracts.shp')
# CT_centroids.set_index('GEOID',inplace=True)
# CT_centroids.drop(columns=['geometry'],inplace=True)


In [36]:
# O_CT_df=CT_0.join(CT_centroids,how='inner')
# O_CT_df.head()

In [37]:
# D_CT_df=pd.read_csv('../Data/Top_10_CBDs.csv',index_col=0).set_index('GEOID').join(CT_centroids,how='left')
# D_CT_df.head()

In [33]:
# D_CT_df['formatted']=D_CT_df.INTPTLAT.astype(str)+"%2C"+D_CT_df.INTPTLON.astype(str)
# D_coords=D_CT_df.formatted.values[0:5]

In [34]:
# destination_coords=""
# for i in D_coords:

#     destination_coords+=i+"%7C"
    
# destination_coords=destination_coords[:-3]
# destination_coords

In [7]:
O_CT_df = CT_0.copy()
O_CT_df['formatted']=O_CT_df.Center_Lat.astype(str)+"%2C"+O_CT_df.Center_Lon.astype(str)
#_coords=D_CT_df.formatted.values[0:5]
O_CT_df.head()

,ctlabel,boro_code,boro_name,ct2010,boro_ct2010,cdeligibil,ntacode,ntaname,puma,shape_leng,shape_area,geometry,Bus_Access,Subway_Access,Bus_Count,Subway_Count,Center_Lat,Center_Lon,area_km2,formatted
geoid,,,,,,,,,,,,,,,,,,,,
36061009800,98.0,1,Manhattan,9800,1009800,I,MN19,Turtle Bay-East Midtown,3808,5534.200308,1.906016e+06,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ...",1,1,22,3,40.755788,-73.967775,0.177075,40.75578797405885%2C-73.96777472301177
36061010200,102.0,1,Manhattan,10200,1010200,I,MN17,Midtown-Midtown South,3807,5687.802439,1.860993e+06,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ...",1,1,25,8,40.759424,-73.974449,0.172893,40.759423723208%2C-73.97444938135632
36061010400,104.0,1,Manhattan,10400,1010400,I,MN17,Midtown-Midtown South,3807,5693.036367,1.864600e+06,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ...",1,1,21,9,40.760779,-73.977673,0.173228,40.760778939480964%2C-73.97767289767248
36061011300,113.0,1,Manhattan,11300,1011300,I,MN17,Midtown-Midtown South,3807,5699.860640,1.890907e+06,"MULTIPOLYGON (((-73.98412 40.75484, -73.98460 ...",1,1,21,14,40.754775,-73.987870,0.175672,40.754775073606%2C-73.98786992507925
36061013000,130.0,1,Manhattan,13000,1013000,I,MN40,Upper East Side-Carnegie Hill,3805,5807.972956,1.918145e+06,"MULTIPOLYGON (((-73.96148 40.77432, -73.96194 ...",1,1,12,2,40.772736,-73.964734,0.178202,40.772735996496955%2C-73.96473354620095


In [8]:
cbd1='40.752251%2C-73.989703' #Midtown/Penn station
cbd2='40.76235%2C-73.98235'  #Midtown North:
cbd3='40.69329%2C-73.98770' #Downtown BK:
cbd4= '40.71433%2C-74.01006' #FIDI
destination_coords=cbd1+'%7C'+cbd2+'%7C'+cbd3+'%7C'+cbd4

In [9]:
destination_coords

'40.752251%2C-73.989703%7C40.76235%2C-73.98235%7C40.69329%2C-73.98770%7C40.71433%2C-74.01006'

In [63]:
# #eXAMPLE CALL

# #9AM on Monday morning
# #for i in ...
# origin_coords='40.7098961%2C-73.9855833'
# url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+origin_coords+"&destinations="+destination_coords+"&units=imperial&arrival_time=1681131600&mode=transit&key="+api_key

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload)

# data = response.json()
# json_data = response.json() if response and response.status_code == 200 else None
# json_data

In [64]:
O_CT_df.index

Int64Index([36061009800, 36061010200, 36061010400, 36061011300, 36061013000,
            36061018400, 36061020600, 36061024900, 36061026900, 36047000501,
            ...
            36081003600, 36047118600, 36047017700, 36047032700, 36005003500,
            36005007300, 36005005100, 36005006300, 36081001900, 36081017100],
           dtype='int64', name='geoid', length=2041)

In [65]:
O_CT_df['CBD1']=np.nan
O_CT_df['CBD2']=np.nan
O_CT_df['CBD3']=np.nan
O_CT_df['CBD4']=np.nan

for geoid in O_CT_df.index.values: #uncomment this out when sure that it works!
    origin_coords=O_CT_df.loc[geoid,'formatted']
    #9AM on Monday morning
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+origin_coords+"&destinations="+destination_coords+"&units=imperial&arrival_time=1681736400&mode=transit&key="+api_key

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    data = response.json()
    json_data = response.json() if response and response.status_code == 200 else None
    try:
        O_CT_df.loc[geoid,'CBD1']=round(json_data['rows'][0]['elements'][0]['duration']['value']/60) #number of minutes it takes to comute
    except:
        O_CT_df.loc[geoid,'CBD1']=np.nan
    try:
        O_CT_df.loc[geoid,'CBD2']=round(json_data['rows'][0]['elements'][1]['duration']['value']/60)
    except:
        O_CT_df.loc[geoid,'CBD2']==np.nan
    try:
        O_CT_df.loc[geoid,'CBD3']=round(json_data['rows'][0]['elements'][2]['duration']['value']/60)
    except:
        O_CT_df.loc[geoid,'CBD3']==np.nan
    try:
        O_CT_df.loc[geoid,'CBD4']=round(json_data['rows'][0]['elements'][3]['duration']['value']/60)
    except:
        O_CT_df.loc[geoid,'CBD4']==np.nan    

In [66]:
response.json()

{'destination_addresses': ['480 Fashion Ave, New York, NY 10018, USA',
  'Manhattan Times Square Hotel 790 7th Ave, corner, W 52nd St, New York, NY 10019, USA',
  '350 Jay St, Brooklyn, NY 11201, USA',
  '54 Murray Street, New York, NY 10007, USA'],
 'origin_addresses': ['39-72 39th St, Queens, NY 11104, USA'],
 'rows': [{'elements': [{'distance': {'text': '5.0 mi', 'value': 8024},
     'duration': {'text': '33 mins', 'value': 1975},
     'status': 'OK'},
    {'distance': {'text': '4.0 mi', 'value': 6460},
     'duration': {'text': '27 mins', 'value': 1620},
     'status': 'OK'},
    {'distance': {'text': '9.2 mi', 'value': 14842},
     'duration': {'text': '46 mins', 'value': 2789},
     'status': 'OK'},
    {'distance': {'text': '7.2 mi', 'value': 11607},
     'duration': {'text': '42 mins', 'value': 2549},
     'status': 'OK'}]}],
 'status': 'OK'}

In [67]:
O_CT_df.head()

,ctlabel,boro_code,boro_name,ct2010,boro_ct2010,cdeligibil,ntacode,ntaname,puma,shape_leng,...,Bus_Count,Subway_Count,Center_Lat,Center_Lon,area_km2,formatted,CBD1,CBD2,CBD3,CBD4
geoid,,,,,,,,,,,,,,,,,,,,,
36061009800,98.0,1,Manhattan,9800,1009800,I,MN19,Turtle Bay-East Midtown,3808,5534.200308,...,22,3,40.755788,-73.967775,0.177075,40.75578797405885%2C-73.96777472301177,19.0,10.0,33.0,27.0
36061010200,102.0,1,Manhattan,10200,1010200,I,MN17,Midtown-Midtown South,3807,5687.802439,...,25,8,40.759424,-73.974449,0.172893,40.759423723208%2C-73.97444938135632,15.0,6.0,30.0,24.0
36061010400,104.0,1,Manhattan,10400,1010400,I,MN17,Midtown-Midtown South,3807,5693.036367,...,21,9,40.760779,-73.977673,0.173228,40.760778939480964%2C-73.97767289767248,15.0,5.0,29.0,25.0
36061011300,113.0,1,Manhattan,11300,1011300,I,MN17,Midtown-Midtown South,3807,5699.860640,...,21,14,40.754775,-73.987870,0.175672,40.754775073606%2C-73.98786992507925,4.0,6.0,26.0,12.0
36061013000,130.0,1,Manhattan,13000,1013000,I,MN40,Upper East Side-Carnegie Hill,3805,5807.972956,...,12,2,40.772736,-73.964734,0.178202,40.772735996496955%2C-73.96473354620095,32.0,19.0,41.0,35.0


In [68]:
O_CT_df.to_csv('../Data/commute_tt_results.csv')